In [ ]:
from hydra.utils import to_absolute_path as abspath
import os

path = abspath("../data/to_predict/")
os.path.isdir(path)

In [ ]:
from pathlib import Path

Path("data/to_predict").is_dir()

In [ ]:
from bs4 import BeautifulSoup
import requests

url = "https://understat.com/league/EPL"

response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

In [ ]:
div = soup.find("div",{"class": "calendar-container"})

In [ ]:
scripts = soup.find_all("script")

In [ ]:
scripts

In [ ]:
import codecs
import json
from pandas import json_normalize

encoded_string = str(scripts[1])
encoded_string = encoded_string.split(
    "JSON.parse('", 1
)[-1]
# encoded_string = encoded_string.rsplit("=",1)[0]
encoded_string = encoded_string.rsplit("');", 1)[0]
# Have it ignore the escape characters so it can decode the ascii
# and be able to use json.loads
jsonStr = codecs.getdecoder("unicode-escape")(
    encoded_string
)[0]
jsonObj = json.loads(jsonStr.split("]")[0]+"]")
df = json_normalize(jsonObj)

In [ ]:
df

In [ ]:
class ScraperFutureMatches:
    def __init__(
        self, 
        leagues = ["EPL", "La liga", "Bundesliga", "Serie A", "Ligue 1", "RFPL"],
        parent_folder = "../data/raw/future_matches"
    ):
        self.leagues = leagues
        self.parent_folder = Path(parent_folder)

    def loop_scrape(
        self
    ):
        # Step 0 - Create parent dir if there isn't one
        if not self.parent_folder.exists():
            self.parent_folder.mkdir()
        
        for league in self.leagues:
            url = (
                    "https://understat.com/league/"
                    + league
                )
            response = requests.get(url)
            soup = BeautifulSoup(response.text, "html.parser")
            scripts = soup.find_all("script")
            for script in scripts:
                if "var datesData" in str(script):
                    encoded_string = str(script)
                    encoded_string = encoded_string.split(
                        "JSON.parse('", 1
                    )[-1]
                    # encoded_string = encoded_string.rsplit("=",1)[0]
                    encoded_string = encoded_string.rsplit("');", 1)[0]
                    # Have it ignore the escape characters so it can decode the ascii
                    # and be able to use json.loads
                    jsonStr = codecs.getdecoder("unicode-escape")(
                        encoded_string
                    )[0]
                    jsonObj = json.loads(jsonStr.split("]")[0]+"]")
                    df = json_normalize(jsonObj)
                    break
            # Step - Save the file
            df.drop(df[df["isResult"] == True].index,inplace=True)
            df["league"] = league
            df.to_csv(self.parent_folder.joinpath(league + ".csv"))

In [ ]:
from pathlib import Path
scraper = ScraperFutureMatches()

In [ ]:
scraper.loop_scrape()